## XOR problem

In [1]:
import torch
import torch.nn as nn

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(DEVICE) # xor data(input, output)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(DEVICE)

In [4]:
class Perceptron(nn.Module):  # 이 perceptron은 정확도가 50%라서 학습이 불가능
    """ 퍼셉트론은 하나의 선형 층입니다 """

    def __init__(self, input_dim):
        """
        매개변수:
            input_dim (int): 입력 특성의 크기
        """
        super(Perceptron, self).__init__()
        self.fc1 = nn.Linear(input_dim, 1)

    def forward(self, x_in):
        """퍼셉트론의 정방향 계산

        매개변수:
            x_in (torch.Tensor): 입력 데이터 텐서
                x_in.shape는 (batch, num_features)입니다.
        반환값:
            결과 텐서. tensor.shape는 (batch,)입니다.
        """
        return torch.sigmoid(self.fc1(x_in))

In [5]:
model = Perceptron(input_dim=2).to(DEVICE)

In [6]:
# model = nn.Sequential(
#           nn.Linear(2, 1, bias=True),
#           nn.Sigmoid()).to(DEVICE)

In [7]:
# linear = nn.Linear(2, 1, bias=True)
# sigmoid = nn.Sigmoid()
# model = nn.Sequential(linear, sigmoid).to(DEVICE)

In [8]:
criterion = torch.nn.BCELoss().to(DEVICE) #loss 계산해주는 함수 (binary cross entropy (logit없이 확률값으로 ))  logit 은 -무한대에서 무한대까지인데, sigmoid취하기 전값
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [9]:
for step in range(10001):
    optimizer.zero_grad() #gradient 초기화
    hypothesis = model(X)

    # 비용 함수
    cost = criterion(hypothesis, Y) #cost=loss
    cost.backward() #gradeint계산
    optimizer.step()#weight에서 gradeint를 뺀다

    if step % 100 == 0: #100번 반복될때마다
        print(step, cost.item()) #loss출력

0 0.7173023223876953
100 0.6969178915023804
200 0.6948033571243286
300 0.6938842535018921
400 0.6934778690338135
500 0.6932963132858276
600 0.6932147741317749
700 0.6931778192520142
800 0.693161129951477
900 0.6931535005569458
1000 0.6931500434875488
1100 0.6931484937667847
1200 0.6931477785110474
1300 0.6931475400924683
1400 0.6931473016738892
1500 0.6931471824645996
1600 0.6931472420692444
1700 0.6931471824645996
1800 0.6931471824645996
1900 0.6931471824645996
2000 0.6931471824645996
2100 0.6931471824645996
2200 0.6931471824645996
2300 0.6931471824645996
2400 0.6931471824645996
2500 0.6931471824645996
2600 0.6931471824645996
2700 0.6931471824645996
2800 0.6931471824645996
2900 0.6931471824645996
3000 0.6931471824645996
3100 0.6931471824645996
3200 0.6931471824645996
3300 0.6931471824645996
3400 0.6931471824645996
3500 0.6931471824645996
3600 0.6931471824645996
3700 0.6931471824645996
3800 0.6931471824645996
3900 0.6931471824645996
4000 0.6931471824645996
4100 0.6931471824645996
4200 

In [10]:
with torch.no_grad(): #밑에서 연산시 gradient가 필요없다
    hypothesis = model(X) #모델에다 입력 넣어서 예측값을 0~1로 출력 hypothesis는 모델 학습완료시 0.5 인데 예측값은 0 또는 1로 바꿈 정확도 0.5는 둘중하나만의 50%는 찍는수준 ,# 이 perceptron은 정확도가 50%라서 학습이 불가능
    predicted = (hypothesis > 0.5).float()  #0~1 예측값을 0또는 1로 바꿈
    accuracy = (predicted == Y).float().mean() #위에서 만든 0 or 1값을 정답과 비교해서 정확도 측정
    print('모델의 출력값(Hypothesis): ', hypothesis.detach().cpu().numpy())
    print('모델의 예측값(Predicted): ', predicted.detach().cpu().numpy())
    print('실제값(Y): ', Y.cpu().numpy())
    print('정확도(Accuracy): ', accuracy.item())

모델의 출력값(Hypothesis):  [[0.5]
 [0.5]
 [0.5]
 [0.5]]
모델의 예측값(Predicted):  [[0.]
 [0.]
 [0.]
 [0.]]
실제값(Y):  [[0.]
 [1.]
 [1.]
 [0.]]
정확도(Accuracy):  0.5


In [11]:
class MLP(nn.Module):
    def __init__(self, input_dim):
        """
        매개변수:
            input_dim (int): 입력 특성의 크기
        """
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 10) #2차원(맨처음 xor->10차원 data input
        self.fc2 = nn.Linear(10, 1) #10차원이 들어가서 1차원으로 나옴

    def forward(self, x_in):
        """퍼셉트론의 정방향 계산

        매개변수:
            x_in (torch.Tensor): 입력 데이터 텐서
                x_in.shape는 (batch, num_features)입니다.
        반환값:
            결과 텐서. tensor.shape는 (batch,)입니다.
        """
        hidden = torch.sigmoid(self.fc1(x_in)) #fully connected지나고 sigmoid지남 ->그래서 10차원 , fc1(x_in)하면 위에 선언한 fc1으로 들어감.그래프상 들어감
        return torch.sigmoid(self.fc2(hidden))

In [12]:
model = MLP(input_dim=2).to(DEVICE) #MLP 클래스의 인스턴스 하나 만듬(코드적), MLP 모델 만들었음(의미적)

In [13]:
# model = nn.Sequential(
#           nn.Linear(2, 10, bias=True), # input_layer = 2, hidden_layer1 = 10
#           nn.Sigmoid(),
#           nn.Linear(10, 10, bias=True), # hidden_layer1 = 10, hidden_layer2 = 10
#           nn.Sigmoid(),
#           nn.Linear(10, 10, bias=True), # hidden_layer2 = 10, hidden_layer3 = 10
#           nn.Sigmoid(),
#           nn.Linear(10, 1, bias=True), # hidden_layer3 = 10, output_layer = 1
#           nn.Sigmoid()
#           ).to(DEVICE)

In [14]:
criterion = torch.nn.BCELoss().to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr=1.0) # stochastic gradient descent(gradient일종), data를 너무 다 하면 시간걸리거나, 메모리가 많아져서, mini batch이요

In [15]:
for epoch in range(10001):
    optimizer.zero_grad()
    # forward 연산
    hypothesis = model(X)

    # 비용 함수
    cost = criterion(hypothesis, Y)
    cost.backward() #gradient 값만 계산부분
    optimizer.step() #weight 업데이트부분(step함수로 계산),

    # 100의 배수에 해당되는 에포크마다 비용을 출력
    if epoch % 100 == 0:
        print(epoch, cost.item())

0 0.6952501535415649
100 0.6916824579238892
200 0.6829251646995544
300 0.5905053615570068
400 0.2778838872909546
500 0.09341105818748474
600 0.04730147495865822
700 0.030291307717561722
800 0.021886274218559265
900 0.016974203288555145
1000 0.013785427436232567
1100 0.011562383733689785
1200 0.009930815547704697
1300 0.008686084300279617
1400 0.007707314565777779
1500 0.00691879540681839
1600 0.006270861253142357
1700 0.0057295579463243484
1800 0.005270920228213072
1900 0.004877720959484577
2000 0.004537051077932119
2100 0.00423919502645731
2200 0.003976720385253429
2300 0.003743696492165327
2400 0.003535573836416006
2500 0.0033486001193523407
2600 0.0031796982511878014
2700 0.003026485675945878
2800 0.0028868471272289753
2900 0.002759132068604231
3000 0.0026418566703796387
3100 0.0025338020641356707
3200 0.002433978021144867
3300 0.0023414622992277145
3400 0.0022554751485586166
3500 0.0021753832697868347
3600 0.0021005996968597174
3700 0.002030657371506095
3800 0.001965086907148361
39

In [16]:
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('모델의 출력값(Hypothesis): ', hypothesis.detach().cpu().numpy())
    print('모델의 예측값(Predicted): ', predicted.detach().cpu().numpy())
    print('실제값(Y): ', Y.cpu().numpy())
    print('정확도(Accuracy): ', accuracy.item())
    # [9.9918824e-01]->9.9918824 * 10^(-1) = 0.9918.. (0에서 1사이에서는 엄청 숫자라서 큰확률임)
    # 3.8756 * 10^(-4) = 3.8756 / 10000 = 0.00038756.. (0에 가까운수)

모델의 출력값(Hypothesis):  [[3.8756276e-04]
 [9.9918824e-01]
 [9.9951267e-01]
 [8.5405010e-04]]
모델의 예측값(Predicted):  [[0.]
 [1.]
 [1.]
 [0.]]
실제값(Y):  [[0.]
 [1.]
 [1.]
 [0.]]
정확도(Accuracy):  1.0
